In [1]:
import os
import glob
import math
import time
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import warnings
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import random
import seaborn as sns; sns.set_theme()
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from matplotlib.pyplot import figure
from IPython import display
from pandas.plotting import scatter_matrix
from sklearn.metrics import r2_score
from sklearn import svm
from numpy import std
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import cm
from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [4]:
train_test_seurat = pd.read_csv('./integrate.csv')

In [5]:
train_test_seurat = train_test_seurat.T

In [6]:
train_test_seurat.reset_index(inplace=True) 
train_test_seurat.drop("index",axis=1,inplace=True)

In [7]:
train_test_seurat_std = train_test_seurat.std()
column_names = list(train_test_seurat.columns)
columns_remove = []
for i in range(train_test_seurat.shape[1]):
    if train_test_seurat_std[i] == 0:
        columns_remove.append(column_names[i])

In [8]:
train_test_seurat = train_test_seurat.drop(columns_remove, axis=1)

In [9]:
train_test_seurat[columns_remove[0]] = train_test_seurat.iloc[:, 0]

In [10]:
train_test_seurat.shape

(109707, 54)

In [11]:
train_test_seurat

,X39,X53,X45,X13,X36,X15,X51,X49,X33,X40,X31,X34,X32,X50,X30,X18,X46,X35,X37,X17,X42,X38,X43,X47,X0,X52,X48,X41,X2,X3,X25,X23,X9,X27,X14,X19,X24,X16,X29,X26,X28,X21,X1,X11,X20,X22,X5,X4,X6,X12,X7,X10,X8,X62
0,-1.582156,1.230171,2.175619,2.246967,-0.755876,3.583055,0.098955,3.040176,0.919099,-0.889421,0.484578,0.481170,0.777112,-1.798247,0.873301,2.270459,2.912388,0.728549,-1.295910,2.275724,-0.359744,-0.363988,-0.717875,2.265473,1.019072,0.721157,2.260484,-1.834228,2.254474,2.254450,3.042155,2.259637,2.946272,2.258885,3.401480,3.345012,2.937407,3.343820,2.935077,2.941428,2.259203,2.938547,2.469231,2.945955,3.044228,2.259368,3.073369,2.943574,2.945749,2.259507,2.259501,3.071520,2.259504,-1.582156
1,-1.584404,1.605045,-2.204271,2.227963,-0.760217,3.535377,0.300338,2.880602,0.943713,-0.890453,0.513255,0.509388,0.803151,-1.042563,0.900408,2.251683,2.831043,0.757081,-1.302729,2.256918,-0.368216,-0.372547,-0.725350,2.242051,1.084322,1.273463,2.239054,-1.836103,2.235786,2.235849,2.995526,2.240825,2.901227,2.240080,3.359956,3.304083,2.898066,3.302884,2.895731,2.902085,2.240400,2.910457,2.446440,2.903884,3.007216,2.240566,3.004351,2.907020,2.906517,2.240683,2.240706,3.025257,2.240732,-1.584404
2,-1.594548,1.172960,2.547328,2.138920,-0.779797,3.321969,-0.241806,2.693446,0.934271,-0.895068,0.505998,0.502247,0.793162,-1.196648,0.893682,2.165409,2.656538,0.750002,-1.334065,2.170502,-0.407147,-0.411875,-0.759699,2.150257,0.909735,0.858274,2.152557,-1.844565,2.150014,2.149977,2.786986,2.154385,2.734335,2.153668,3.172415,3.119236,2.720591,3.118004,2.718235,2.724605,2.154003,2.731983,2.340769,2.729945,2.798645,2.154173,2.832534,2.727687,2.728807,2.154322,2.154294,2.816114,2.154304,-1.594548
3,-1.540117,1.471016,-1.464351,2.589700,-0.674728,4.499712,-0.070470,3.630669,-0.163725,-0.869522,0.771819,0.763805,1.057894,-1.013944,1.144550,2.601021,3.586176,1.014063,-1.175844,2.588606,-0.210578,-0.213300,-0.586264,2.586420,1.452937,1.030784,2.588699,-1.799160,2.583419,2.583483,3.941371,2.590836,3.676733,2.589976,4.174776,4.107204,3.672950,4.106144,3.670702,3.676991,2.590239,3.685219,2.879247,3.681201,3.949020,2.590387,3.953846,3.682269,3.681686,2.590519,2.590540,3.971859,2.590523,-1.540117
4,-1.601161,1.122772,2.557003,2.079653,-0.792563,3.184293,0.169903,2.380626,0.968678,-0.898042,0.546909,0.542502,0.829559,-1.363513,-0.469419,2.107699,2.530198,0.790754,-1.355026,2.112698,-0.433188,-0.438182,-0.782676,2.091026,0.925432,0.645674,2.091058,-1.850081,2.092565,2.092535,2.652600,2.096564,2.618092,2.095867,2.443792,2.998522,2.604873,2.997269,2.602504,2.608884,2.096211,2.615890,2.270373,2.613631,2.663541,2.096384,2.693932,2.611509,2.613064,2.096538,2.096525,2.682193,2.096533,-1.601161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109702,-1.811006,-2.112556,-0.131305,-1.445311,-1.197634,-0.818321,-1.049270,-1.122315,-0.508777,-0.357198,1.027011,-0.311548,0.721451,-2.989994,0.889711,-1.483919,-1.102665,-0.136718,0.872560,-1.485440,0.397786,-2.070832,0.489914,-1.491222,-4.525414,-2.391811,-1.488305,0.216442,-1.483623,-1.483645,-1.030355,-1.481442,-1.076202,-1.483214,-1.062814,-1.047183,-1.082408,-1.057290,-1.070795,-1.077186,-1.483106,-1.077155,-1.348169,-1.074557,-1.005825,-1.483853,-0.998264,-1.075246,-1.074719,-1.483446,-1.483451,-1.000564,-1.483459,-1.811006
109703,0.753172,-1.087454,-0.172401,-0.671626,0.416608,-0.605980,-0.603986,-0.787102,1.405494,-0.960323,0.122744,1.482535,0.217549,-1.818208,-0.006719,-0.725304,-0.736942,1.349400,0.214743,-0.727687,-1.691140,0.832361,-1.892583,-0.731506,-3.272527,-1.266335,-0.731930,0.941092,-0.720566,-0.720532,-0.645284,-0.724744,-0.706717,-0.723241,-0.704043,-0.710849,-0.699841,-0.697351,-0.719490,-0.712830,-0.725970,-0.712226,-0.739703,-0.709396,-0.687081,-0.725374,-0.687902,-0.708704,-0.709225,-0.725369,-0.725369,-0.684054,-0.725367,0.753172
1

In [12]:
train_seurat = train_test_seurat.iloc[:90000, :]
test_seurat = train_test_seurat.iloc[90000:, :]

test_seurat.reset_index(inplace=True) 
test_seurat.drop("index",axis=1,inplace=True)

In [13]:
train = pd.read_csv('./MLR_Project_train.csv')
test = pd.read_csv('./MLR_Project_test.csv')
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]

In [14]:
train_seurat['TARGET'] = train['TARGET']
test_seurat['TARGET'] = test['TARGET']

In [15]:
test_seurat

,X39,X53,X45,X13,X36,X15,X51,X49,X33,X40,X31,X34,X32,X50,X30,X18,X46,X35,X37,X17,X42,X38,X43,X47,X0,X52,X48,X41,X2,X3,X25,X23,X9,X27,X14,X19,X24,X16,X29,X26,X28,X21,X1,X11,X20,X22,X5,X4,X6,X12,X7,X10,X8,X62,TARGET
0,0.303619,-1.368098,0.318033,-0.664706,0.031642,-0.637868,-0.901035,-0.789375,-0.896748,-0.626164,-0.752216,-1.400652,-0.490283,-2.085998,-0.213036,-0.730901,-0.721203,-1.097359,1.065540,-0.730536,0.097683,0.585017,-0.309425,-0.733860,0.162955,-1.752634,-0.736883,0.265702,-0.732184,-0.732188,-0.691366,-0.726183,-0.704597,-0.728807,-0.710549,-0.708777,-0.706969,-0.709911,-0.709945,-0.706522,-0.728111,-0.709008,-0.754861,-0.704447,-0.684198,-0.728199,-0.680463,-0.704870,-0.704561,-0.728003,-0.728004,-0.679034,-0.728009,0.303619,0.010438
1,0.270544,-0.293297,-0.800283,0.205806,-0.817904,-0.267324,-0.575346,-0.088519,0.252945,0.516988,-0.888595,-0.183459,-0.612266,-1.281293,-0.365916,0.108370,-0.104701,0.079553,0.531337,0.111701,1.092659,0.558323,0.561124,0.114858,0.637649,-0.454719,0.113503,1.172467,0.110364,0.110365,-0.143167,0.113187,-0.092878,0.115930,-0.129771,-0.128735,-0.094559,-0.137471,-0.090028,-0.090889,0.114014,-0.093737,0.036367,-0.092026,-0.128428,0.113681,-0.124515,-0.092511,-0.091235,0.113804,0.113818,-0.122337,0.113839,0.270544,-0.000532
2,0.408199,-1.086199,0.476146,-0.632996,0.711765,-0.619817,-0.865759,-0.691751,-0.650123,-0.491680,-0.751901,-1.105154,-0.523756,-1.759521,-1.664908,-0.695644,-0.693250,-0.846007,1.020084,-0.695285,0.317099,0.571099,-0.093730,-0.691992,-0.162791,-1.178735,-0.693527,0.322890,-0.696694,-0.696709,-0.664141,-0.691275,-0.680123,-0.693348,-0.691434,-0.689171,-0.681219,-0.686197,-0.688039,-0.683237,-0.692878,-0.683230,-0.722643,-0.680831,-0.662410,-0.692527,-0.657241,-0.680773,-0.681192,-0.692505,-0.692521,-0.657914,-0.692516,0.408199,0.000726
3,0.320914,-0.331991,-0.839812,0.407444,0.304629,-0.155761,-0.566030,0.025930,-1.156736,1.424197,-0.719807,-0.181216,-0.422551,-0.963558,-0.236426,0.301783,0.072858,0.096051,0.556588,0.303111,1.065361,0.716041,0.727413,0.300229,0.981367,-0.507419,0.303556,1.175421,0.300083,0.300104,0.026371,0.305106,0.084091,0.308407,0.038148,0.039061,0.082331,0.028317,0.089581,0.086617,0.305908,0.083850,0.227740,0.085438,0.038418,0.305550,0.042203,0.085026,0.085652,0.305663,0.305680,0.043227,0.305671,0.320914,0.001459
4,0.399412,-0.842829,0.232809,-0.624319,1.072002,-0.617945,0.000683,-0.669520,-0.532747,-0.602045,-0.789767,-0.971547,-0.547541,-1.815948,-1.508678,-0.686499,-0.687388,-0.725842,1.004514,-0.686138,0.421749,0.566900,0.010782,-0.680960,-0.271713,-1.155744,-0.683218,0.399896,-0.688086,-0.688104,-0.661634,-0.682309,-0.675430,-0.684194,-0.686846,-0.684348,-0.676160,-0.681855,-0.682763,-0.678181,-0.683735,-0.678193,-0.715838,-0.676215,-0.658180,-0.683386,-0.653732,-0.675658,-0.676009,-0.683393,-0.683402,-0.653591,-0.683384,0.399412,-0.001462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19702,-1.811006,-2.112556,-0.131305,-1.445311,-1.197634,-0.818321,-1.049270,-1.122315,-0.508777,-0.357198,1.027011,-0.311548,0.721451,-2.989994,0.889711,-1.483919,-1.102665,-0.136718,0.872560,-1.485440,0.397786,-2.070832,0.489914,-1.491222,-4.525414,-2.391811,-1.488305,0.216442,-1.483623,-1.483645,-1.030355,-1.481442,-1.076202,-1.483214,-1.062814,-1.047183,-1.082408,-1.057290,-1.070795,-1.077186,-1.483106,-1.077155,-1.348169,-1.074557,-1.005825,-1.483853,-0.998264,-1.075246,-1.074719,-1.483446,-1.483451,-1.000564,-1.483459,-1.811006,0.000061
19703,0.753172,-1.087454,-0.172401,-0.671626,0.416608,-0.605980,-0.603986,-0.787102,1.405494,-0.960323,0.122744,1.482535,0.217549,-1.818208,-0.006719,-0.725304,-0.736942,1.349400,0.214743,-0.727687,-1.691140,0.832361,-1.892583,-0.731506,-3.272527,-1.266335,-0.731930,0.941092,-0.720566,-0.720532,-0.645284,-0.724744,-0.706717,-0.723241,-0.704043,-0.7108

In [16]:
train_max = np.sum(train['TARGET'][train['TARGET']>0])
test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Maximum return of training set:', train_max)
print('Maximum return of testing set:', test_max)

Maximum return of training set: 195.6927566509
Maximum return of testing set: 55.96225182400002


In [17]:
reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seurat.iloc[:, :-1]), train_seurat['TARGET'])
train_pred = reg.predict(pd.DataFrame(train_seurat.iloc[:, :-1]))

test_pred = reg.predict(pd.DataFrame(test_seurat.iloc[:, :-1]))

train_res = np.sum(train['TARGET'][train_pred>0])
test_res = np.sum(test['TARGET'][test_pred>0])

In [18]:
print(f'Train naive random selection percentage return: {train_res/train_max*100}%')
print(f'Test naive random selection percentage return: {test_res/test_max*100}%')

Train naive random selection percentage return: 6.326525261528151%
Test naive random selection percentage return: 4.010962118285209%


In [19]:
train = train_seurat
test = test_seurat

In [20]:
train

,X39,X53,X45,X13,X36,X15,X51,X49,X33,X40,X31,X34,X32,X50,X30,X18,X46,X35,X37,X17,X42,X38,X43,X47,X0,X52,X48,X41,X2,X3,X25,X23,X9,X27,X14,X19,X24,X16,X29,X26,X28,X21,X1,X11,X20,X22,X5,X4,X6,X12,X7,X10,X8,X62,TARGET
0,-1.582156,1.230171,2.175619,2.246967,-0.755876,3.583055,0.098955,3.040176,0.919099,-0.889421,0.484578,0.481170,0.777112,-1.798247,0.873301,2.270459,2.912388,0.728549,-1.295910,2.275724,-0.359744,-0.363988,-0.717875,2.265473,1.019072,0.721157,2.260484,-1.834228,2.254474,2.254450,3.042155,2.259637,2.946272,2.258885,3.401480,3.345012,2.937407,3.343820,2.935077,2.941428,2.259203,2.938547,2.469231,2.945955,3.044228,2.259368,3.073369,2.943574,2.945749,2.259507,2.259501,3.071520,2.259504,-1.582156,0.013314
1,-1.584404,1.605045,-2.204271,2.227963,-0.760217,3.535377,0.300338,2.880602,0.943713,-0.890453,0.513255,0.509388,0.803151,-1.042563,0.900408,2.251683,2.831043,0.757081,-1.302729,2.256918,-0.368216,-0.372547,-0.725350,2.242051,1.084322,1.273463,2.239054,-1.836103,2.235786,2.235849,2.995526,2.240825,2.901227,2.240080,3.359956,3.304083,2.898066,3.302884,2.895731,2.902085,2.240400,2.910457,2.446440,2.903884,3.007216,2.240566,3.004351,2.907020,2.906517,2.240683,2.240706,3.025257,2.240732,-1.584404,-0.000448
2,-1.594548,1.172960,2.547328,2.138920,-0.779797,3.321969,-0.241806,2.693446,0.934271,-0.895068,0.505998,0.502247,0.793162,-1.196648,0.893682,2.165409,2.656538,0.750002,-1.334065,2.170502,-0.407147,-0.411875,-0.759699,2.150257,0.909735,0.858274,2.152557,-1.844565,2.150014,2.149977,2.786986,2.154385,2.734335,2.153668,3.172415,3.119236,2.720591,3.118004,2.718235,2.724605,2.154003,2.731983,2.340769,2.729945,2.798645,2.154173,2.832534,2.727687,2.728807,2.154322,2.154294,2.816114,2.154304,-1.594548,0.000244
3,-1.540117,1.471016,-1.464351,2.589700,-0.674728,4.499712,-0.070470,3.630669,-0.163725,-0.869522,0.771819,0.763805,1.057894,-1.013944,1.144550,2.601021,3.586176,1.014063,-1.175844,2.588606,-0.210578,-0.213300,-0.586264,2.586420,1.452937,1.030784,2.588699,-1.799160,2.583419,2.583483,3.941371,2.590836,3.676733,2.589976,4.174776,4.107204,3.672950,4.106144,3.670702,3.676991,2.590239,3.685219,2.879247,3.681201,3.949020,2.590387,3.953846,3.682269,3.681686,2.590519,2.590540,3.971859,2.590523,-1.540117,-0.000628
4,-1.601161,1.122772,2.557003,2.079653,-0.792563,3.184293,0.169903,2.380626,0.968678,-0.898042,0.546909,0.542502,0.829559,-1.363513,-0.469419,2.107699,2.530198,0.790754,-1.355026,2.112698,-0.433188,-0.438182,-0.782676,2.091026,0.925432,0.645674,2.091058,-1.850081,2.092565,2.092535,2.652600,2.096564,2.618092,2.095867,2.443792,2.998522,2.604873,2.997269,2.602504,2.608884,2.096211,2.615890,2.270373,2.613631,2.663541,2.096384,2.693932,2.611509,2.613064,2.096538,2.096525,2.682193,2.096533,-1.601161,0.003811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,0.103760,-0.323209,-0.033912,0.504407,0.932129,-0.098634,-0.687340,0.052372,0.175746,-0.657793,-1.040164,-0.270910,-0.719094,-0.871932,-0.530427,0.405426,0.159104,-0.021451,0.626506,0.407638,1.068273,0.466155,0.542083,0.404342,0.637946,-0.699337,0.402187,1.206836,0.403753,0.403796,0.111838,0.409407,0.165668,0.412404,0.118302,0.112101,0.166753,0.107222,0.167445,0.164019,0.409890,0.160662,0.327742,0.165710,0.112724,0.409590,0.121034,0.165462,0.165721,0.409705,0.409703,0.115440,0.409704,0.103760,0.001185
89996,0.270911,-1.178868,-0.215649,-0.526271,-0.831779,-0.589190,-0.853648,-0.699739,-0.942459,-0.612974,-0.772829,-1.455120,-0.514554,-1.595189,-0.289078,-0.589598,-0.636953,-1.152731,1.088975,-0.589015,0.058042,0.642626,-0.349257,-0.591799,-0.636469,-1.493630,-0.593950,0.298781,-0.592508,-0.592574,-0.619011,-0.584295,-0.627197,-0.587219,-0.635496,-0.635929,-0.626819,-0.635882,-0.631618,-0.629097,-0.586609,-0.630493,-0.639392,-0.627132,-0.614351,-0.586671,-0.614544,-0.626798,-0.627063,-0.586522,-0.58

In [21]:
train_ = pd.DataFrame()

for i in range(train.shape[1]-1):
    for j in range(train.shape[1]-1):
        train_[str(i)+'_'+str(j)+'_feat'] = train.iloc[:, i] * train.iloc[:, j]
        
train_target = pd.DataFrame(train['TARGET'])

train = train.drop(['TARGET'], axis = 1)

train = pd.concat([train, train_], axis = 1)

train = (train-train.mean())/train.std()

train['TARGET'] = train_target

In [22]:
test_ = pd.DataFrame()

for i in range(test.shape[1]-1):
    for j in range(test.shape[1]-1):
        test_[str(i)+'_'+str(j)+'_feat'] = test.iloc[:, i] * test.iloc[:, j]
        
test_target = pd.DataFrame(test['TARGET'])

test = test.drop(['TARGET'], axis = 1)

test = pd.concat([test, test_], axis = 1)

test = (test-test.mean())/test.std()

test['TARGET'] = test_target

## 5.5 Autoencoder Resnet model

In [23]:
input_features = train.drop(['TARGET'], axis=1).to_numpy()
output_features = pd.DataFrame((np.sign(train['TARGET'])+1)//2).to_numpy()
# output_features = train['TARGET'].to_numpy()

X_test = test.drop(['TARGET'], axis=1).to_numpy()
Y_test = pd.DataFrame((np.sign(test['TARGET'])+1)//2).to_numpy()
# Y_test = test['TARGET'].to_numpy()

In [24]:
input_features_mean = np.mean(input_features, axis = 1)
input_features_mean = input_features_mean.reshape((input_features_mean.shape[0], 1))
input_features = input_features- input_features_mean

In [25]:
X_test_mean = np.mean(X_test, axis = 1)
X_test_mean = X_test_mean.reshape((X_test_mean.shape[0], 1))
X_test = X_test - X_test_mean

In [26]:
# X_train, X_val, Y_train, Y_val = train_test_split(input_features, output_features, test_size=0.2, random_state=42)

X_train = X_val = input_features
Y_train = Y_val = output_features

####
# train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)
train_data = val_data = train
test_data = test
####

auto_train_max = np.sum(train_data['TARGET'][train_data['TARGET']>0])
auto_val_max = np.sum(val_data['TARGET'][val_data['TARGET']>0])
auto_test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Train X shape:', X_train.shape)
print('Validation X shape:', X_val.shape)
print('Test X shape:', X_test.shape)

print('Train Y shape:', Y_train.shape)
print('Val Y shape:', Y_val.shape)
print('Test Y shape:', Y_test.shape)

print('train_max:', auto_train_max)
print('val_max:', auto_val_max)
print('test_max:', auto_test_max)

Train X shape: (90000, 2970)
Validation X shape: (90000, 2970)
Test X shape: (19707, 2970)
Train Y shape: (90000, 1)
Val Y shape: (90000, 1)
Test Y shape: (19707, 1)
train_max: 195.6927566509
val_max: 195.6927566509
test_max: 55.96225182400002


In [27]:
train_input = torch.from_numpy(X_train)
train_output = torch.from_numpy(Y_train)
val_input = torch.from_numpy(X_val)
val_output = torch.from_numpy(Y_val)
test_input = torch.from_numpy(X_test)
test_output = torch.from_numpy(Y_test)

# train_input = torch.unsqueeze(train_input, 1)
# val_input = torch.unsqueeze(val_input, 1)
# test_input = torch.unsqueeze(test_input, 1)

train_input = train_input.float()
train_output = train_output.float()
val_input = val_input.float()
val_output = val_output.float()
test_input = test_input.float()
test_output = test_output.float()

input_feature = train_input.shape[1]
output_feature = 1

print('input_feature:', input_feature)
print('output_feature:', output_feature)

input_feature: 2970
output_feature: 1


In [28]:
train_input = train_input.to(device)
train_output = train_output.to(device)
val_input = val_input.to(device)
val_output = val_output.to(device)
test_input = test_input.to(device)
test_output = test_output.to(device)

In [29]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [30]:
# auto-encoder model
# base model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.linear1 = nn.Linear(input_feature, input_feature*2)
        self.linear2 = nn.Linear(input_feature*2, input_feature//16)
        self.linear3 = nn.Linear(input_feature//4, input_feature//16)
        self.linear4 = nn.Linear(input_feature//16, input_feature//16)
        
        self.linear5 = nn.Linear(input_feature//16, input_feature//16)
        self.linear6 = nn.Linear(input_feature//16, input_feature//16)
        
        self.batchnorm_1 = nn.BatchNorm1d(input_feature//2)
        self.batchnorm_2 = nn.BatchNorm1d(input_feature//4)
        self.batchnorm_3 = nn.BatchNorm1d(input_feature//16)
        self.linear = nn.Linear(input_feature//16, 2)
        
        # nn.init.constant_(self.linear1.weight, 0.1)
        # nn.init.constant_(self.linear2.weight, 0.1)
        # nn.init.constant_(self.linear3.weight, 0.1)
        # nn.init.constant_(self.linear4.weight, 0.1)
        # nn.init.constant_(self.linear.weight, 0.1)
        self.relu = nn.ReLU()
#         self.leakyrelu = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(0.15)
        
        self.softmax = nn.Softmax()
        

    def forward(self, x):
        x = self.linear1(x)
#         x = self.batchnorm_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.linear2(x)
#         x = self.batchnorm_2(x)
        x = self.relu(x)
#         x = self.dropout(x)
        
        output = self.linear(x)
                
        return output.float()

In [31]:
batch_size = 100000
train_ds = TensorDataset(train_input, train_output)
train_dl = DataLoader(train_ds, batch_size= batch_size, shuffle=False)

In [32]:
%matplotlib inline
def fit(num_epochs, model, loss_fn, train_input, train_output, val_input, val_output, test_input, test_output, model_path):
    best_loss = float('inf')
    train_pred_output = []
    val_pred_output = []
    train_error = []
    val_error = []
    test_error = []
    epochs = []
    
    train_returns = []
    val_returns = []
    test_returns = []
    
    train_sum = []
    val_sum = []
    test_sum = []

    for epoch in range(num_epochs):
        for x,y in train_dl:
            model = model.train()
            opt.zero_grad()
            pred = model(x)
            # y = torch.reshape(y, (y.shape[0], 1))
            loss = loss_fn(pred, y.long().squeeze())
            loss.backward()
            opt.step()

        if epoch % 500 == 0:
            
            model = model.eval()
            
            train_pred = model(train_input)
            train_output_index = (torch.sign(train_output)+1)//2
            train_pred_index = (torch.sign(train_pred)+1)//2
            train_output = torch.reshape(train_output, (train_output_index.shape[0], 1))
            # train_loss = loss_fn(train_output, train_pred)
            train_loss = loss_fn(train_pred, train_output.long().squeeze())
            train_loss = train_loss.cpu().detach().numpy()
            
            val_pred = model(val_input)
            val_pred_index = (torch.sign(val_pred)+1)//2
            val_output = torch.reshape(val_output, (val_output.shape[0], 1))
            # val_loss = loss_fn(val_output, val_pred)
            val_loss = loss_fn(val_pred, val_output.long().squeeze())
            val_loss = val_loss.cpu().detach().numpy()
        
            test_pred = model(test_input)
            test_pred_index = (torch.sign(test_pred)+1)//2
            test_output = torch.reshape(test_output, (test_output.shape[0], 1))
            # test_loss = loss_fn(test_output, test_pred)
            test_loss = loss_fn(test_pred, test_output.long().squeeze())
            test_loss = test_loss.cpu().detach().numpy()
    
            epochs.append(epoch)
            train_error.append(math.log(train_loss+1))
            val_error.append(math.log(val_loss+1))
            test_error.append(math.log(test_loss+1))
            
#             figure, ax = plt.subplots(1, 2, figsize = (20, 7))
#             ax = ax.flatten()
            
#             figure, ax = plt.subplots(1, 4, figsize = (22, 5))
#             ax = ax.flatten()
            
#             plt.grid(False)
            # train_conf = confusion_matrix(train_output.cpu().detach().numpy(), train_pred_index.cpu().detach().numpy())
#             g1 = sns.heatmap(train_conf, cmap="YlGnBu",cbar=False, ax=ax[0], annot = True)
#             g1.set_ylabel('True Target')
#             g1.set_xlabel('Predict Target')
#             g1.set_title('Train dataset')

#             plt.grid(False)
            # val_conf = confusion_matrix(val_output.cpu().detach().numpy(), val_pred_index.cpu().detach().numpy())
#             g2 = sns.heatmap(val_conf, cmap="YlGnBu",cbar=False, ax=ax[1], annot = True)
#             g2.set_ylabel('True Target')
#             g2.set_xlabel('Predict Target')
#             g2.set_title('Val dataset')
            
#             plt.grid(False)
            # test_conf = confusion_matrix(test_output.cpu().detach().numpy(), test_pred_index.cpu().detach().numpy())
#             g3 = sns.heatmap(test_conf, cmap="YlGnBu",cbar=False, ax=ax[2], annot = True)
#             g3.set_ylabel('True Target')
#             g3.set_xlabel('Predict Target')
#             g3.set_title('Test dataset')
            
            
            softmax = nn.Softmax(dim=1)
            train_pred_np = torch.argmax(softmax(train_pred), 1)
            val_pred_np = torch.argmax(softmax(val_pred), 1)
            test_pred_np = torch.argmax(softmax(test_pred), 1)
            # print(train_pred_np)
                            
            # train_pred_np = train_pred_index.cpu().detach().numpy()
            train_output_np = train_output.cpu().detach().numpy()
            # val_pred_np = val_pred_index.cpu().detach().numpy()
            val_output_np = val_output.cpu().detach().numpy()
            # test_pred_np = test_pred_index.cpu().detach().numpy()
            test_output_np = test_output.cpu().detach().numpy()
            
#             train_max_value = max(max(train_output_np), max(train_pred_np))
#             train_min_value = min(min(train_output_np), min(train_pred_np))
#             val_max_value = max(max(val_output_np), max(val_pred_np))
#             val_min_value = min(min(val_output_np), min(val_pred_np))
#             test_max_value = max(max(test_output_np), max(test_pred_np))
#             test_min_value = min(min(test_output_np), min(test_pred_np))
            
#             ax[0].scatter(train_output_np, train_pred_np, s = 20, alpha=0.3, c='blue')
#             ax[1].scatter(val_output_np, val_pred_np, s = 20, alpha=0.3, c='red')
#             ax[2].scatter(test_output_np, test_pred_np, s = 20, alpha=0.3, c='green')
            
#             ax[0].plot(epochs, train_error, c='blue')
#             ax[0].plot(epochs, val_error, c='red')
#             ax[0].plot(epochs, test_error, c='green')
#             ax[0].set_title('Errors vs Epochs', fontsize=15)
#             ax[0].set_xlabel('Epoch', fontsize=10)
#             ax[0].set_ylabel('Errors', fontsize=10)

#             ax[0].legend(['train', 'valid', 'test'])
            
#             ax[0].set_xlim([train_min_value, train_max_value])
#             ax[0].set_ylim([train_min_value, train_max_value])
#             ax[0].set_title('Trainig data', fontsize=15)
#             ax[0].set_xlabel('Target', fontsize=10)
#             ax[0].set_ylabel('Prediction', fontsize=10)
#             ax[0].plot([train_min_value, train_max_value], [train_min_value, train_max_value], 'k-')
            
#             ax[1].set_xlim([val_min_value, val_max_value])
#             ax[1].set_ylim([val_min_value, val_max_value])
#             ax[1].set_title('Validation data', fontsize=15)
#             ax[1].set_xlabel('Target', fontsize=10)
#             ax[1].set_ylabel('Prediction', fontsize=10)
#             ax[1].plot([val_min_value, val_max_value], [val_min_value, val_max_value], 'k-')
            
#             ax[2].set_xlim([test_min_value, test_max_value])
#             ax[2].set_ylim([test_min_value, test_max_value])
#             ax[2].set_title('Testing data', fontsize=15)
#             ax[2].set_xlabel('Target', fontsize=10)
#             ax[2].set_ylabel('Prediction', fontsize=10)
#             ax[2].plot([test_min_value, test_max_value], [test_min_value, test_max_value], 'k-')
            
#             ax[3].plot(epochs, train_error, c='blue')
#             ax[3].plot(epochs, val_error, c='red')
#             ax[3].plot(epochs, test_error, c='green')
#             ax[3].set_title('Training and Validation error', fontsize=15)
#             ax[3].set_xlabel('Epochs', fontsize=10)
#             ax[3].set_ylabel('MSE error', fontsize=10)
            
#             display.clear_output(wait=True)
#             display.display(pl.gcf())
            
#             print('Epoch ', epoch, 'Train_loss: ', train_loss*1000, ' Validation_loss: ', val_loss*100, ' Test_loss: ', test_loss*100)
            # print(train_pred_np.shape, train_pred_np)
            # print(train_pred, train_pred_np)
            # train_pred_np = np.squeeze(train_pred_np)
            # val_pred_np = np.squeeze(val_pred_np)
            # test_pred_np = np.squeeze(test_pred_np)
            train_pred_np = train_pred_np.cpu().detach().numpy()
            val_pred_np = val_pred_np.cpu().detach().numpy()
            test_pred_np = test_pred_np.cpu().detach().numpy()
            
            train_res = np.sum(train_data['TARGET'][train_pred_np>0])
            train_output_check = np.squeeze(train_output_np)
            train_check = np.sum(train_data['TARGET'][train_output_check>0])
            
            val_res = np.sum(val_data['TARGET'][val_pred_np>0])
            val_output_check = np.squeeze(val_output_np)
            val_check = np.sum(val_data['TARGET'][val_output_check>0])
            
            test_res = np.sum(test_data['TARGET'][test_pred_np>0])
            test_output_check = np.squeeze(test_output_np)
            test_check = np.sum(test_data['TARGET'][test_output_check>0])
            
#             train_returns.append(train_res)
#             val_returns.append(val_res)
#             test_returns.append(test_res)
            
#             ax[1].plot(epochs, train_returns, c='blu`e')
#             ax[1].plot(epochs, val_returns, c='red')
#             ax[1].plot(epochs, test_returns, c='green')
#             ax[1].legend(['train', 'valid', 'test'])
#             ax[1].set_title('Return vs Epochs', fontsize=15)
#             ax[1].set_xlabel('Epoch', fontsize=10)
#             ax[1].set_ylabel('Returns', fontsize=10)

#             display.clear_output(wait=True)
#             display.display(pl.gcf())
            
            train_sum.append(train_res)
            val_sum.append(val_res)
            test_sum.append(test_res)
            # print(f'Checks: {train_check/auto_train_max*100}%, {val_check/auto_val_max*100}%, {test_check/auto_test_max*100}%')
#             print(f'Maximum sum train return {train_res}, Total train return: {auto_train_max}, Maximum train percentage return: {train_res/auto_train_max*100}%')
#             print(f'Maximum sum train return {val_res}, Total train return: {auto_val_max}, Maximum train percentage return: {val_res/auto_val_max*100}%')
#             print(f'Maximum sum test return {test_res}, Total test return: {auto_test_max}, Maximum test percentage return: {test_res/auto_test_max*100}%')
#             print('Epoch:', epoch, 'Train loss:', train_loss, 'Val loss:', val_loss, 'Test loss:', test_loss)
            print(f'Epoch: {epoch}, Train loss: {train_loss}, Train return: {train_res/auto_train_max*100}%, Val loss: {val_loss}, Val return: {val_res/auto_val_max*100}%, Test loss: {test_loss}, Test return: {test_res/auto_test_max*100}%')
            # print(np.squeeze(train_output.cpu().detach().numpy()))
            # print(train_pred_np)
            # print(confusion_matrix(np.squeeze(train_output.cpu().detach().numpy()), train_pred_np))
            # print(confusion_matrix(np.squeeze(val_output.cpu().detach().numpy()), val_pred_np))
            # print(confusion_matrix(np.squeeze(test_output.cpu().detach().numpy()), test_pred_np))
            # print(train_conf)
            # print(val_conf)
            # print(test_conf)
            # print(train_output, train_pred)
            # if val_loss < best_loss:
            #     torch.save(model.state_dict(), model_path)
            #     best_loss = val_loss
                
#             train_pred_output.append([train_pred.cpu().detach().numpy(), train_output.cpu().detach().numpy()])
#             val_pred_output.append([val_pred.cpu().detach().numpy(), val_output.cpu().detach().numpy()])
    return train_sum, val_sum, test_sum


In [33]:
num_epochs = 5000
learning_rate = 0.001
# loss_fn = F.mse_loss
loss_fn = nn.CrossEntropyLoss()

seed_everything()

model = Autoencoder()
model = model.to(device)
opt = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
train_sum_1, val_sum_1, test_sum_1 = fit(num_epochs, model, loss_fn, train_input, train_output, val_input, val_output, test_input, test_output, 'model_path_seurat_classsify')

Epoch: 0, Train loss: 0.6955394744873047, Train return: -1.0641574543379546%, Val loss: 0.6955394744873047, Val return: -1.0641574543379546%, Test loss: 0.6950516700744629, Test return: -0.6055377436664738%
Epoch: 500, Train loss: 0.6917592287063599, Train return: 2.9345145906674532%, Val loss: 0.6917592287063599, Val return: 2.9345145906674532%, Test loss: 0.6930858492851257, Test return: 0.26887408046624756%
Epoch: 1000, Train loss: 0.6910677552223206, Train return: 4.214865951944307%, Val loss: 0.6910677552223206, Val return: 4.214865951944307%, Test loss: 0.6930437684059143, Test return: -0.07565629977353451%
Epoch: 1500, Train loss: 0.6904795169830322, Train return: 4.841359555786318%, Val loss: 0.6904795169830322, Val return: 4.841359555786318%, Test loss: 0.6930810213088989, Test return: -0.7782185505502005%
Epoch: 2000, Train loss: 0.6899368762969971, Train return: 5.569457669576901%, Val loss: 0.6899368762969971, Val return: 5.569457669576901%, Test loss: 0.6931464672088623, T

KeyboardInterrupt: 